# 五分钟快速入门(Five-minute quickstart)  

https://materialsproject.github.io/fireworks/quickstart.html;

## 启动

1. 启动MongoDB: `mongod.exe --nojournal --dbpath .`

2. 重置/初始化 FireWork发射台(LaunchPad): `lpad reset`    
    * `lpad`(-h)打开FireWorks的命令行界面, 并显示帮助信息;
    * `lpad -version`: 查看FireWorks的版本信息和安装目录: `c:\users\tfliu\anaconda3\lib\site-packages`;

## 添加工作流

为工作流添加两个脚本(作业,步骤), 将这两个脚本命名为`hello`和`goodbye`, 并将该工作流命名为`test_workflow`:    
`lpad add_scripts 'echo "hello"' 'echo "goodbye"' -n hello goodbye -w test_workflow`

查看工作流信息(工作流执行之前):  
`lpad get_wflows -n test_workflow -d more`

## 运行所有工作流程

运行工作流中的单个作业, 或者所有工作: `rlaunch --silencer rapidfire`   
工作流运行完后再次查看工作流信息: `lpad get_wflows -n test_workflow -d more`

FireWorks会为工作流中的每个作业创建目录(cmd: user/tfliu/launcher_blk, python: 'pwd')并运行它们;    
[可以选择运行作业的位置](https://materialsproject.github.io/fireworks/controlworker.html), 以及是否在[运行作业后拆除空目录](https://materialsproject.github.io/fireworks/config_tutorial.html);

## 启动

启动web浏览器查看结果: `lpad webgui`

## Python code

In [8]:
from fireworks import Firework, Workflow, LaunchPad, ScriptTask
from fireworks.core.rocket_launcher import rapidfire

In [9]:
# 新建发射台并且初始化
launchpad = LaunchPad()
launchpad.reset('', require_password=False)

2018-10-26 02:46:29,066 INFO Performing db tune-up
2018-10-26 02:46:29,084 INFO LaunchPad was RESET.


In [10]:
# FireWorks: 创建作业;
fw1 = Firework(ScriptTask.from_str('echo "hello"'), name="hello")
fw2 = Firework(ScriptTask.from_str('echo "goodbye"'), name="goodbye")
print(fw2)

Firework object: (id: -4 , name: fireworks::Firework)


In [11]:
# Workflow: 创建工作流;
wf = Workflow([fw1, fw2], {fw1:fw2}, name="test workflow")
print(wf)

Workflow object: (fw_ids: dict_keys([-3, -4]) , name: test workflow)


In [12]:
%%pass
fw1 = Firework(ScriptTask.from_str('echo "hello"'), name="hello")
fw2 = Firework(ScriptTask.from_str('echo "goodbye"'), name="goodbye", parents=[fw1])
wf = Workflow([fw1, fw2], name="test workflow")

In [13]:
# 将工作流加入向发射台
launchpad.add_wf(wf)

2018-10-26 02:46:29,336 INFO Added a workflow. id_map: {-4: 1, -3: 2}


{-4: 1, -3: 2}

In [14]:
# 发射所有任务(开火)
rapidfire(launchpad)

2018-10-26 02:46:29,410 INFO Created new dir Y:\Jupyter\work\fireworks\launcher_2018-10-25-18-46-29-410775
2018-10-26 02:46:29,412 INFO Launching Rocket
2018-10-26 02:46:29,432 INFO RUNNING fw_id: 2 in directory: Y:\Jupyter\work\fireworks\launcher_2018-10-25-18-46-29-410775
2018-10-26 02:46:29,447 INFO Task started: ScriptTask.
2018-10-26 02:46:29,494 INFO Task completed: ScriptTask 
2018-10-26 02:46:29,513 INFO Rocket finished
2018-10-26 02:46:29,545 INFO Created new dir Y:\Jupyter\work\fireworks\launcher_2018-10-25-18-46-29-518897
2018-10-26 02:46:29,547 INFO Launching Rocket
2018-10-26 02:46:29,565 INFO RUNNING fw_id: 1 in directory: Y:\Jupyter\work\fireworks\launcher_2018-10-25-18-46-29-518897
2018-10-26 02:46:29,578 INFO Task started: ScriptTask.
2018-10-26 02:46:29,620 INFO Task completed: ScriptTask 
2018-10-26 02:46:29,642 INFO Rocket finished


# 入门教程(Introductory Tutorial)

## 在Python下使用fireworks

在当前目录创建结果文件

In [25]:
from fireworks import Firework, LaunchPad, ScriptTask
from fireworks.core.rocket_launcher import launch_rocket

# set up the LaunchPad and reset it
launchpad = LaunchPad()
launchpad.reset('', require_password=False)

# create the Firework consisting of a single task
firetask = ScriptTask.from_str('echo "howdy, your job launched successfully!"')
firework = Firework(firetask)

# store workflow and launch it locally
launchpad.add_wf(firework)
launch_rocket(launchpad)

2018-10-26 03:02:45,869 INFO Performing db tune-up
2018-10-26 03:02:45,878 INFO LaunchPad was RESET.
2018-10-26 03:02:45,882 INFO Added a workflow. id_map: {-6: 1}
2018-10-26 03:02:45,884 INFO Launching Rocket
2018-10-26 03:02:45,896 INFO RUNNING fw_id: 1 in directory: Y:\Jupyter\work\fireworks
2018-10-26 03:02:45,943 INFO Task started: ScriptTask.
2018-10-26 03:02:45,974 INFO Task completed: ScriptTask 
2018-10-26 03:02:45,987 INFO Rocket finished


True

## 一些有用的修改

更改MongoDB配置

In [27]:
%%pass
launchpad = LaunchPad(host="myhost", port=12345, name="fireworks_testing_db", 
                      username="my_user", password="my_pass")

以快速模式运行

In [28]:
%%pass
from fireworks import Firework, LaunchPad, ScriptTask, FWorker # use this line instead of the first line
from fireworks.core.rocket_launcher import rapidfire
rapidfire(launchpad, FWorker())  # use this line instead of launch_rocket()

仅输出警告

In [29]:
%%pass
launchpad = LaunchPad(strm_lvl='WARNING')
# <code omitted>
launch_rocket(launchpad, FWorker(), strm_lvl='WARNING')

将工作流写出到平面文件(flat file), 或从文件加载Firework对象;

In [30]:
%%pass
fw_yaml = firework.to_file("my_firework.yaml")
fw = firework.from_file("my_firework.yaml")
print(fw)

In [31]:
%%pass
fw_json = firework.to_file("my_firework.json")
fw = firework.from_file("my_firework.json")
print(fw)